In [2]:
import os
import json
import pandas as pd
import streamlit as st
import st_pages as stp

from os.path import exists

base_url = "https://trisonics-scouting-api.azurewebsites.net/api"
from scout import *

2024-03-13 13:08:07.007 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-03-13 13:08:07.009 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-03-13 13:08:07.011 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-03-13 13:08:07.013 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-03-13 13:08:07.015 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-03-13 13:08:07.017 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [5]:
sk = '4003data'
ek = '2024milac'

event = load_event_data(sk, ek)

2024-03-12 20:06:18.507 
  command:

    streamlit run /home/jbuist/git/FRCScout/data-streamlit/venv/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-03-12 20:06:18.509 No runtime found, using MemoryCacheStorageManager


https://trisonics-scouting-api.azurewebsites.net/api/GetResults?secret_team_key=4003data&event_key=2024milac


In [84]:
opr = load_opr_data(sk, ek)
# opr[opr['teamNumber'] == 4003]['totalPoints'].item()
opr_totalpoints = opr[['teamNumber', 'totalPoints']]

https://trisonics-scouting-api.azurewebsites.net/api/GetOPRData?secret_team_key=4003data&event_key=2024milac


In [30]:
opr.columns

Index(['adjustPoints', 'autoAmpNoteCount', 'autoAmpNotePoints',
       'autoLeavePoints', 'autoPoints', 'autoSpeakerNoteCount',
       'autoSpeakerNotePoints', 'autoTotalNotePoints', 'endGameHarmonyPoints',
       'endGameNoteInTrapPoints', 'endGameOnStagePoints', 'endGameParkPoints',
       'endGameSpotLightBonusPoints', 'endGameTotalStagePoints',
       'ensembleBonusOnStageRobotsThreshold',
       'ensembleBonusStagePointsThreshold', 'foulCount', 'foulPoints',
       'melodyBonusThreshold', 'melodyBonusThresholdCoop',
       'melodyBonusThresholdNonCoop', 'rp', 'techFoulCount',
       'teleopAmpNoteCount', 'teleopAmpNotePoints', 'teleopPoints',
       'teleopSpeakerNoteAmplifiedCount', 'teleopSpeakerNoteAmplifiedPoints',
       'teleopSpeakerNoteCount', 'teleopSpeakerNotePoints',
       'teleopTotalNotePoints', 'totalPoints', 'teamNumber'],
      dtype='object')

In [31]:
matches = load_matches_data(ek)

2024-03-12 22:58:54.750 No runtime found, using MemoryCacheStorageManager


https://trisonics-scouting-api.azurewebsites.net/api/GetMatchesForEvent?event_key=2024milac


In [98]:
n1 = pd.json_normalize(matches['alliances'])
bluen2 = pd.json_normalize(n1['blue.team_keys'])
redn2 = pd.json_normalize(n1['red.team_keys'])


blueteams = pd.DataFrame(n1['blue.team_keys'].explode())
redteams = pd.DataFrame(n1['red.team_keys'].explode())
blueteams['blue.team_keys'] = blueteams['blue.team_keys'].str.replace('frc', '').astype(int)
redteams['red.team_keys'] = redteams['red.team_keys'].str.replace('frc', '').astype(int)
blueteams['robot_num'] = blueteams.groupby(blueteams.index).rank()
redteams['robot_num'] = redteams.groupby(redteams.index).rank()
pblue = blueteams.pivot(columns='robot_num', values='blue.team_keys')
pred = redteams.pivot(columns='robot_num', values='red.team_keys')
pblue.columns = ['blue1', 'blue2', 'blue3']
pred.columns = ['red1', 'red2', 'red3']
bigmatches = matches.join(pblue).join(pred)


In [88]:
matches.columns

Index(['actual_time', 'alliances', 'comp_level', 'event_key', 'key',
       'match_number', 'post_result_time', 'predicted_time', 'score_breakdown',
       'set_number', 'time', 'videos', 'winning_alliance'],
      dtype='object')

In [ ]:
for color, index in zip(['blue', 'red'], [1, 2, 3]):
    bigmatches = bigmatches.join(opr_totalpoints.set_index('teamNumber'), on=f'{color}{index}')
    bigmatches.rename(columns={'totalPoints': f'{color}{index}_totalPoints'}, inplace=True)
    bigmatches = bigmatches.join(opr_totalpoints.set_index('teamNumber'), on=f'{color}{index}')
    bigmatches.rename(columns={'totalPoints': f'{color}{index}_totalPoints'}, inplace=True)
bigmatches

In [105]:
from scout import load_statbot_matches_data

predictions = load_statbot_matches_data(ek)

ImportError: cannot import name 'load_statbot_matches_data' from 'scout' (/home/jbuist/git/FRCScout/data-streamlit/scout.py)

In [3]:
from scout import load_statbot_matches_data
statbotics = load_statbot_matches_data('2024milac')
preds = statbotics[['comp_level', 'match_number', 'pred']]


2024-03-13 13:08:14.648 
  command:

    streamlit run /home/jbuist/git/FRCScout/data-streamlit/venv/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-03-13 13:08:14.652 No runtime found, using MemoryCacheStorageManager


statbot url https://trisonics-scouting-api.azurewebsites.net/api/GetStatboticsMatches?event_key=2024milac


In [8]:
import pandas as pd

bigpreds = preds.join(pd.json_normalize(preds['pred']))
bigpreds

,comp_level,match_number,pred,winner,red_win_prob,red_score,blue_score,red_melody_rp,red_harmony_rp,blue_melody_rp,blue_harmony_rp
0,f,1,"{'winner': 'blue', 'red_win_prob': 0.3386, 're...",blue,0.3386,59.25,65.63,0.5468,0.2469,0.8694,0.0658
1,f,2,"{'winner': 'blue', 'red_win_prob': 0.3524, 're...",blue,0.3524,59.85,65.52,0.5632,0.2469,0.8721,0.0658
2,f,3,"{'winner': 'blue', 'red_win_prob': 0.342400000...",blue,0.3424,59.69,65.56,0.5576,0.2469,0.8750,0.0658
3,qm,1,"{'winner': 'blue', 'red_win_prob': 0.440600000...",blue,0.4406,46.64,48.71,0.3075,0.0911,0.4396,0.1341
4,qm,10,"{'winner': 'red', 'red_win_prob': 0.5135000000...",red,0.5135,44.89,44.41,0.2986,0.1087,0.2748,0.1210
...,...,...,...,...,...,...,...,...,...,...,...
89,sf,1,"{'winner': 'red', 'red_win_prob': 0.6463, 'red...",red,0.6463,58.89,51.06,0.6369,0.0373,0.3199,0.2097
90,sf,1,"{'winner': 'blue', 'red_win_prob': 0.0882, 're...",blue,0.0882,46.01,65.70,0.2112,0.1246,0.8726,0.1651
91,sf,1,"{'winner': 'blue', 'red_win_prob': 0.464400000...",blue,0.4644,57.31,58.58,0.6615,0.1226,0.5609,0.2469
92,sf,1,"{'winner': 'red', 'red_win_prob': 0.7604000000...",red,0.7604,63.16,52.36,0.8727,0.0658,0.2935,0.2613
